In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as preprocessing 
from sklearn.decomposition import PCA
import tensorflow as tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from sklearn.metrics import confusion_matrix

# Load MNIST Dataset

In [98]:
# Training Data
Train = pd.read_csv('/home/tonzhu/Downloads/mnist_train.csv', header = None)
print(Train.shape)
Train.head(10)

(60000, 785)


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
Train_data = Train.iloc[:,1:]
Train_label_or = Train.iloc[:,0]

In [99]:
# Test Data
Test = pd.read_csv('/home/tonzhu/Downloads/mnist_test.csv', header = None)
print(Test.shape)
Test.head(10)

(10000, 785)


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
Test_data = Test.iloc[:,1:]
Test_label_or = Test.iloc[:,0]

# Data Preprocessing

In [60]:
# Normalization (min_max, make number between 0 and 1)
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True).fit(Train_data)
X_train_minmax = minmax_scale.transform(Train_data)
X_test_minmax = minmax_scale.transform(Test_data)

In [102]:
# Copy training data for shuffle (Original Data)
X_df = Train_data.copy()
X_df['label'] = Train_label
X_df2 = X_df.copy()

In [ ]:
# Copy training data for shuffle (Normalized Data)
X_copy = X_train_minmax.copy()
X_df = pd.DataFrame(X_copy)
X_df['label'] = Train_label
X_df2 = X_df.copy()

In [103]:
# Training 
Train_data_or = Train_data.copy().values
Test_data_or = Test_data.copy().values

In [84]:
# Training 
Train_data_or = X_train_minmax.copy()
Test_data_or = X_test_minmax.copy()

In [63]:
# Create One-hot encoding for labels
Train_label_oh_or = pd.get_dummies(Train_label).values
Test_label_oh_or = pd.get_dummies(Test_label).values

In [64]:
tf.set_random_seed(1)
np.random.seed(1)

# CNN Training

In [156]:
# Construct CNN architecture
tf.reset_default_graph()

# Constants
training_epochs = 50
batch_size = 100
total_batch = int(len(Train) / batch_size)
learning_rate = 0.001 
display_step = 10

# Variables
x = tf.placeholder(tf.float32, [None, 28*28])
y = tf.placeholder(tf.int32, [None, 10])            # input y
image = tf.reshape(x, [-1, 28, 28, 1])              # (batch, height, width, channel)

# CNN
conv1 = tf.layers.conv2d(   # shape (28, 28, 1)
    inputs = image,
    filters = 32,
    kernel_size = 5,
    strides = 1,
    padding = 'same',
    activation = tf.nn.relu
) # (28, 28, 32)
pool1 = tf.layers.max_pooling2d(
    conv1,
    pool_size=2,
    strides=2,
) # (14, 14, 32)
conv2 = tf.layers.conv2d(pool1, 64, 5, 1, 'same', activation=tf.nn.relu)  # (14, 14, 64)
pool2 = tf.layers.max_pooling2d(conv2, 2, 2)    # (7, 7, 64)
flat = tf.reshape(pool2, [-1, 7*7*64])          # (7*7*64, )
fc = tf.layers.dropout(flat, rate = 0.5)
output = tf.layers.dense(fc, 10)              # output
decode_label = tf.argmax(output, axis = 1)

# Loss function, optimizer and results
loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=output) 
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
accuracy = tf.metrics.accuracy(labels=tf.argmax(y, axis=1), predictions=tf.argmax(output, axis=1),)[1] # get accuracy

# Initializing
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

# Training and Test
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    step = 1
    for epoch in xrange(training_epochs):
        # Transfer training data to 28 x 28 size
        Train_data = X_df2.iloc[:,:-1].values
        Train_label = X_df2.iloc[:,-1]
        Train_label_oh = pd.get_dummies(Train_label).values
        for i in xrange(total_batch):
            batch_x = Train_data[i * batch_size : (i + 1) * batch_size,:]
            batch_y = Train_label_oh[i * batch_size : (i + 1) * batch_size,:]
            sess.run(optimizer, feed_dict = {x: batch_x, y: batch_y})
            if step % display_step == 0:
                acc = sess.run(accuracy, feed_dict = {x: batch_x, y: batch_y})
                cost = sess.run(loss, feed_dict = {x: batch_x, y: batch_y})
                print  ('Iter ' + str(step * batch_size) + ': Minibatch Loss = ' +
                       '{: .6f}'.format(cost) + ', Training Accuracy = ' +
                       '{: .5f}'.format(acc))
            step += 1
        # Shuffle Data
        X_df2 = X_df2.sample(frac = 1).reset_index(drop = True)
    print ('Optimization Finished!')
    # Training, Test Results
    train_predict = np.array([])
    for i in xrange(total_batch):
        part_x = Train_data_or[i * batch_size : (i + 1) * batch_size]
        train_predict = np.append(train_predict, sess.run(decode_label, {x: part_x}))
    test_predict = np.array([])
    for i in xrange(total_batch):
        part_x_test = Test_data_or[i * batch_size : (i + 1) * batch_size]
        test_predict = np.append(test_predict, sess.run(decode_label, {x: part_x_test}))
            

Iter 1000: Minibatch Loss =  4.703124, Training Accuracy =  0.38000
Iter 2000: Minibatch Loss =  0.967083, Training Accuracy =  0.53500
Iter 3000: Minibatch Loss =  0.618033, Training Accuracy =  0.63667
Iter 4000: Minibatch Loss =  0.503067, Training Accuracy =  0.69000
Iter 5000: Minibatch Loss =  0.324288, Training Accuracy =  0.73600
Iter 6000: Minibatch Loss =  0.340461, Training Accuracy =  0.75833
Iter 7000: Minibatch Loss =  0.334536, Training Accuracy =  0.77714
Iter 8000: Minibatch Loss =  0.124886, Training Accuracy =  0.80250
Iter 9000: Minibatch Loss =  0.330131, Training Accuracy =  0.81444
Iter 10000: Minibatch Loss =  0.253616, Training Accuracy =  0.82500
Iter 11000: Minibatch Loss =  0.128455, Training Accuracy =  0.83818
Iter 12000: Minibatch Loss =  0.234814, Training Accuracy =  0.84583
Iter 13000: Minibatch Loss =  0.131620, Training Accuracy =  0.85462
Iter 14000: Minibatch Loss =  0.145130, Training Accuracy =  0.86143
Iter 15000: Minibatch Loss =  0.062178, Tra

Iter 120000: Minibatch Loss =  0.011687, Training Accuracy =  0.96800
Iter 121000: Minibatch Loss =  0.018813, Training Accuracy =  0.96826
Iter 122000: Minibatch Loss =  0.013793, Training Accuracy =  0.96852
Iter 123000: Minibatch Loss =  0.028078, Training Accuracy =  0.96862
Iter 124000: Minibatch Loss =  0.015475, Training Accuracy =  0.96887
Iter 125000: Minibatch Loss =  0.022988, Training Accuracy =  0.96904
Iter 126000: Minibatch Loss =  0.060136, Training Accuracy =  0.96921
Iter 127000: Minibatch Loss =  0.006415, Training Accuracy =  0.96945
Iter 128000: Minibatch Loss =  0.002592, Training Accuracy =  0.96969
Iter 129000: Minibatch Loss =  0.013234, Training Accuracy =  0.96992
Iter 130000: Minibatch Loss =  0.014130, Training Accuracy =  0.97008
Iter 131000: Minibatch Loss =  0.004298, Training Accuracy =  0.97031
Iter 132000: Minibatch Loss =  0.066632, Training Accuracy =  0.97045
Iter 133000: Minibatch Loss =  0.023971, Training Accuracy =  0.97053
Iter 134000: Minibat

Iter 238000: Minibatch Loss =  0.056309, Training Accuracy =  0.98034
Iter 239000: Minibatch Loss =  0.017210, Training Accuracy =  0.98042
Iter 240000: Minibatch Loss =  0.003416, Training Accuracy =  0.98050
Iter 241000: Minibatch Loss =  0.018811, Training Accuracy =  0.98058
Iter 242000: Minibatch Loss =  0.011263, Training Accuracy =  0.98066
Iter 243000: Minibatch Loss =  0.002043, Training Accuracy =  0.98074
Iter 244000: Minibatch Loss =  0.007915, Training Accuracy =  0.98082
Iter 245000: Minibatch Loss =  0.015694, Training Accuracy =  0.98086
Iter 246000: Minibatch Loss =  0.010546, Training Accuracy =  0.98093
Iter 247000: Minibatch Loss =  0.000771, Training Accuracy =  0.98101
Iter 248000: Minibatch Loss =  0.003468, Training Accuracy =  0.98109
Iter 249000: Minibatch Loss =  0.006022, Training Accuracy =  0.98116
Iter 250000: Minibatch Loss =  0.022590, Training Accuracy =  0.98124
Iter 251000: Minibatch Loss =  0.010112, Training Accuracy =  0.98131
Iter 252000: Minibat

Iter 356000: Minibatch Loss =  0.072980, Training Accuracy =  0.98598
Iter 357000: Minibatch Loss =  0.006763, Training Accuracy =  0.98602
Iter 358000: Minibatch Loss =  0.020406, Training Accuracy =  0.98603
Iter 359000: Minibatch Loss =  0.010043, Training Accuracy =  0.98607
Iter 360000: Minibatch Loss =  0.008216, Training Accuracy =  0.98611
Iter 361000: Minibatch Loss =  0.016048, Training Accuracy =  0.98615
Iter 362000: Minibatch Loss =  0.003271, Training Accuracy =  0.98619
Iter 363000: Minibatch Loss =  0.010622, Training Accuracy =  0.98620
Iter 364000: Minibatch Loss =  0.002073, Training Accuracy =  0.98624
Iter 365000: Minibatch Loss =  0.001667, Training Accuracy =  0.98627
Iter 366000: Minibatch Loss =  0.001236, Training Accuracy =  0.98631
Iter 367000: Minibatch Loss =  0.016622, Training Accuracy =  0.98632
Iter 368000: Minibatch Loss =  0.004970, Training Accuracy =  0.98636
Iter 369000: Minibatch Loss =  0.006833, Training Accuracy =  0.98640
Iter 370000: Minibat

Iter 474000: Minibatch Loss =  0.009289, Training Accuracy =  0.98886
Iter 475000: Minibatch Loss =  0.007998, Training Accuracy =  0.98888
Iter 476000: Minibatch Loss =  0.068616, Training Accuracy =  0.98889
Iter 477000: Minibatch Loss =  0.011798, Training Accuracy =  0.98891
Iter 478000: Minibatch Loss =  0.025686, Training Accuracy =  0.98891
Iter 479000: Minibatch Loss =  0.007042, Training Accuracy =  0.98894
Iter 480000: Minibatch Loss =  0.010223, Training Accuracy =  0.98896
Iter 481000: Minibatch Loss =  0.002378, Training Accuracy =  0.98898
Iter 482000: Minibatch Loss =  0.003097, Training Accuracy =  0.98900
Iter 483000: Minibatch Loss =  0.004662, Training Accuracy =  0.98903
Iter 484000: Minibatch Loss =  0.013435, Training Accuracy =  0.98903
Iter 485000: Minibatch Loss =  0.001639, Training Accuracy =  0.98905
Iter 486000: Minibatch Loss =  0.003693, Training Accuracy =  0.98907
Iter 487000: Minibatch Loss =  0.001796, Training Accuracy =  0.98910
Iter 488000: Minibat

...

Iter 2455000: Minibatch Loss =  0.000000, Training Accuracy =  0.99721
Iter 2456000: Minibatch Loss =  0.000000, Training Accuracy =  0.99721
Iter 2457000: Minibatch Loss =  0.000000, Training Accuracy =  0.99722
Iter 2458000: Minibatch Loss =  0.000000, Training Accuracy =  0.99722
Iter 2459000: Minibatch Loss =  0.000000, Training Accuracy =  0.99722
Iter 2460000: Minibatch Loss =  0.000000, Training Accuracy =  0.99722
Iter 2461000: Minibatch Loss =  0.000000, Training Accuracy =  0.99722
Iter 2462000: Minibatch Loss =  0.000000, Training Accuracy =  0.99722
Iter 2463000: Minibatch Loss =  0.000000, Training Accuracy =  0.99722
Iter 2464000: Minibatch Loss =  0.000000, Training Accuracy =  0.99722
Iter 2465000: Minibatch Loss =  0.000000, Training Accuracy =  0.99723
Iter 2466000: Minibatch Loss =  0.000000, Training Accuracy =  0.99723
Iter 2467000: Minibatch Loss =  0.000000, Training Accuracy =  0.99723
Iter 2468000: Minibatch Loss =  0.000000, Training Accuracy =  0.99723
Iter 2

Iter 2571000: Minibatch Loss =  0.000000, Training Accuracy =  0.99732
Iter 2572000: Minibatch Loss =  0.000007, Training Accuracy =  0.99732
Iter 2573000: Minibatch Loss =  0.000000, Training Accuracy =  0.99732
Iter 2574000: Minibatch Loss =  0.000000, Training Accuracy =  0.99732
Iter 2575000: Minibatch Loss =  0.000000, Training Accuracy =  0.99732
Iter 2576000: Minibatch Loss =  0.000000, Training Accuracy =  0.99733
Iter 2577000: Minibatch Loss =  0.000173, Training Accuracy =  0.99733
Iter 2578000: Minibatch Loss =  0.000000, Training Accuracy =  0.99733
Iter 2579000: Minibatch Loss =  0.000000, Training Accuracy =  0.99733
Iter 2580000: Minibatch Loss =  0.000000, Training Accuracy =  0.99733
Iter 2581000: Minibatch Loss =  0.000000, Training Accuracy =  0.99733
Iter 2582000: Minibatch Loss =  0.000000, Training Accuracy =  0.99733
Iter 2583000: Minibatch Loss =  0.000000, Training Accuracy =  0.99733
Iter 2584000: Minibatch Loss =  0.000000, Training Accuracy =  0.99733
Iter 2

Iter 2687000: Minibatch Loss =  0.396438, Training Accuracy =  0.99741
Iter 2688000: Minibatch Loss =  0.000000, Training Accuracy =  0.99741
Iter 2689000: Minibatch Loss =  0.000000, Training Accuracy =  0.99741
Iter 2690000: Minibatch Loss =  0.000000, Training Accuracy =  0.99741
Iter 2691000: Minibatch Loss =  0.000000, Training Accuracy =  0.99741
Iter 2692000: Minibatch Loss =  0.000000, Training Accuracy =  0.99741
Iter 2693000: Minibatch Loss =  0.000209, Training Accuracy =  0.99742
Iter 2694000: Minibatch Loss =  0.000000, Training Accuracy =  0.99742
Iter 2695000: Minibatch Loss =  0.000000, Training Accuracy =  0.99742
Iter 2696000: Minibatch Loss =  0.000000, Training Accuracy =  0.99742
Iter 2697000: Minibatch Loss =  0.000000, Training Accuracy =  0.99742
Iter 2698000: Minibatch Loss =  0.000000, Training Accuracy =  0.99742
Iter 2699000: Minibatch Loss =  0.000000, Training Accuracy =  0.99742
Iter 2700000: Minibatch Loss =  0.072768, Training Accuracy =  0.99742
Iter 2

Iter 2803000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2804000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2805000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2806000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2807000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2808000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2809000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2810000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2811000: Minibatch Loss =  0.000000, Training Accuracy =  0.99751
Iter 2812000: Minibatch Loss =  0.000921, Training Accuracy =  0.99751
Iter 2813000: Minibatch Loss =  0.000000, Training Accuracy =  0.99752
Iter 2814000: Minibatch Loss =  0.000000, Training Accuracy =  0.99752
Iter 2815000: Minibatch Loss =  0.000000, Training Accuracy =  0.99752
Iter 2816000: Minibatch Loss =  0.000035, Training Accuracy =  0.99752
Iter 2

Iter 2919000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2920000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2921000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2922000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2923000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2924000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2925000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2926000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2927000: Minibatch Loss =  0.000001, Training Accuracy =  0.99761
Iter 2928000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2929000: Minibatch Loss =  0.000000, Training Accuracy =  0.99761
Iter 2930000: Minibatch Loss =  0.000021, Training Accuracy =  0.99761
Iter 2931000: Minibatch Loss =  0.000000, Training Accuracy =  0.99762
Iter 2932000: Minibatch Loss =  0.000000, Training Accuracy =  0.99762
Iter 2

In [157]:
accuracy_CNN = float(np.trace(confusion_matrix(Train_label_or.values, train_predcit))) / float(len(Train))

In [158]:
accuracy_CNN

0.9807333333333333

In [159]:
accuracy_CNN_test = float(np.trace(confusion_matrix(Test_label_or.values, test_predcit))) / float(len(Test))

In [160]:
accuracy_CNN_test

0.979